ZOMATO BANGALORE RESTAURANT ANALYSIS - COMPREHENSIVE DATA ANALYTICS PROJECT
=================================================================================

Author: Data Analytics Portfolio Project
Dataset: Bangalore Restaurants from Zomato
Objective: Perform end-to-end data analysis with geospatial insights

Skills Demonstrated:
- Data Cleaning & Transformation
- Exploratory Data Analysis (EDA)
- Statistical Analysis
- Geospatial Data Preparation
- Feature Engineering
- Business Intelligence & Insights

Tools: Python, Pandas, NumPy, Matplotlib, Seaborn, Geopy
=================================================================================

In [11]:
# SECTION 1: Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
from collections import Counter

# Visualization settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [33]:
print("STEP 1: LOADING DATA")
print("-" * 80)

df = pd.read_excel('resturant.xlsx',engine='openpyxl')

print(f"✓ Dataset loaded successfully")
print(f"  → Total Records: {len(df):,}")
print(f"  → Total Features: {df.shape[1]}")
print(f"\nDataset Shape: {df.shape}")
print(f"\nColumn Names:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")
print(df.head())

STEP 1: LOADING DATA
--------------------------------------------------------------------------------
✓ Dataset loaded successfully
  → Total Records: 51,717
  → Total Features: 17

Dataset Shape: (51717, 17)

Column Names:
  1. url
  2. address
  3. name
  4. online_order
  5. book_table
  6. rate
  7. votes
  8. phone
  9. location
  10. rest_type
  11. dish_liked
  12. cuisines
  13. approx_cost(for two people)
  14. reviews_list
  15. menu_item
  16. listed_in(type)
  17. listed_in(city)
                                                 url  \
0  https://www.zomato.com/bangalore/jalsa-banasha...   
1  https://www.zomato.com/bangalore/spice-elephan...   
2  https://www.zomato.com/SanchurroBangalore?cont...   
3  https://www.zomato.com/bangalore/addhuri-udupi...   
4  https://www.zomato.com/bangalore/grand-village...   

                                             address                   name  \
0  942, 21st Main Road, 2nd Stage, Banashankari, ...                  Jalsa   
1  2nd F

In [35]:
print("STEP 2: DATA QUALITY ASSESSMENT")
print("-" * 80)

# Check for missing values
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Percentage': (df.isnull().sum().values / len(df) * 100).round(2)
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

print("\nMissing Values Summary:")
print(missing_data.to_string(index=False))

# Data types
print("\n\nData Types:")
print(df.dtypes)

# Duplicate check
duplicates = df.duplicated().sum()
print(f"\n\nDuplicate Rows: {duplicates}")

STEP 2: DATA QUALITY ASSESSMENT
--------------------------------------------------------------------------------

Missing Values Summary:
                     Column  Missing_Count  Missing_Percentage
                 dish_liked          28078               54.29
                       rate           7775               15.03
                      phone           1208                2.34
approx_cost(for two people)            346                0.67
                  rest_type            227                0.44
                   cuisines             45                0.09
                   location             21                0.04


Data Types:
url                             object
address                         object
name                            object
online_order                    object
book_table                      object
rate                            object
votes                            int64
phone                           object
location                        

In [36]:
print("STEP 3: DATA CLEANING & TRANSFORMATION")
print("-" * 80)

# Create a copy for cleaning
df_clean = df.copy()

print("Cleaning Restaurant Names...")
def clean_restaurant_name(name):
    """Remove special characters and normalize restaurant names"""
    if pd.isna(name):
        return name
    # Remove special encoded characters
    name = re.sub(r'Ã[^\s]*', '', str(name))
    # Remove extra spaces
    name = ' '.join(name.split())
    return name.strip()

df_clean['name'] = df_clean['name'].apply(clean_restaurant_name)
print("  ✓ Restaurant names cleaned")

# -------- 4.2: Clean and Convert Ratings --------
print("\n4.2 Processing Rating Column...")
def clean_rating(rate):
    """Convert rating string to float"""
    if pd.isna(rate) or rate in ['NEW', '-', 'nan']:
        return np.nan
    try:
        # Extract numeric part (e.g., '4.1/5' -> 4.1)
        return float(str(rate).split('/')[0].strip())
    except:
        return np.nan

df_clean['rate_numeric'] = df_clean['rate'].apply(clean_rating)
print(f"  ✓ Ratings converted to numeric")
print(f"  → Valid ratings: {df_clean['rate_numeric'].notna().sum():,}")
print(f"  → Missing ratings: {df_clean['rate_numeric'].isna().sum():,}")


STEP 3: DATA CLEANING & TRANSFORMATION
--------------------------------------------------------------------------------
Cleaning Restaurant Names...
  ✓ Restaurant names cleaned

4.2 Processing Rating Column...
  ✓ Ratings converted to numeric
  → Valid ratings: 41,665
  → Missing ratings: 10,052


In [39]:
#-------- 4.3: Clean Cost Column --------
print("\n4.3 Processing Cost Column...")
df_clean['cost_for_two'] = pd.to_numeric(df_clean['approx_cost(for two people)'], errors='coerce')
print(f"  ✓ Cost converted to numeric")
print(f"  → Valid cost entries: {df_clean['cost_for_two'].notna().sum():,}")

# -------- 4.4: Clean Location Names --------
print("\n4.4 Cleaning Location Names...")
df_clean['location'] = df_clean['location'].str.strip()
print(f"  ✓ Location names cleaned")
print(f"  → Unique locations: {df_clean['location'].nunique()}")




4.3 Processing Cost Column...
  ✓ Cost converted to numeric
  → Valid cost entries: 51,371

4.4 Cleaning Location Names...
  ✓ Location names cleaned
  → Unique locations: 93


In [41]:
# -------- 4.5: Process Cuisine Column --------
print("\n4.5 Processing Cuisines...")
df_clean['cuisine_count'] = df_clean['cuisines'].str.split(',').str.len()
df_clean['primary_cuisine'] = df_clean['cuisines'].str.split(',').str[0].str.strip()
print(f"  ✓ Cuisine features created")
print(f"  → Average cuisines per restaurant: {df_clean['cuisine_count'].mean():.2f}")

# -------- 4.6: Binary Features --------
print("\n4.6 Converting Binary Features...")
df_clean['has_online_order'] = (df_clean['online_order'] == 'Yes').astype(int)
df_clean['has_table_booking'] = (df_clean['book_table'] == 'Yes').astype(int)
print(f"  ✓ Binary features created")



4.5 Processing Cuisines...
  ✓ Cuisine features created
  → Average cuisines per restaurant: 2.45

4.6 Converting Binary Features...
  ✓ Binary features created


In [43]:

# -------- 4.7: Extract Phone Numbers --------
print("\n4.7 Processing Phone Numbers...")
df_clean['phone_count'] = df_clean['phone'].str.split('\n').str.len()
df_clean['has_phone'] = df_clean['phone'].notna().astype(int)
print(f"  ✓ Phone features extracted")

# -------- 4.8: Process Dish Liked --------
print("\n4.8 Processing Popular Dishes...")
def extract_dishes(dish_str):
    """Extract list of dishes from comma-separated string"""
    if pd.isna(dish_str):
        return []
    return [d.strip() for d in str(dish_str).split(',')]

df_clean['dishes_list'] = df_clean['dish_liked'].apply(extract_dishes)
df_clean['dish_count'] = df_clean['dishes_list'].apply(len)
print(f"  ✓ Dish features created")
print(f"  → Restaurants with dish data: {(df_clean['dish_count'] > 0).sum():,}")

print("\n" + "="*80)
print("DATA CLEANING COMPLETE")
print("="*80)
print(f"\nCleaned Dataset Shape: {df_clean.shape}")
print(f"Ready for Analysis: {df_clean.shape[0]:,} restaurants")


4.7 Processing Phone Numbers...
  ✓ Phone features extracted

4.8 Processing Popular Dishes...
  ✓ Dish features created
  → Restaurants with dish data: 23,639

DATA CLEANING COMPLETE

Cleaned Dataset Shape: (51717, 27)
Ready for Analysis: 51,717 restaurants


In [55]:
print("STEP 4: EXPLORATORY DATA ANALYSIS")
print("-" * 80)

# -------- 5.1: Overall Statistics --------
print("\n5.1 Overall Dataset Statistics")
print("-" * 40)

stats = {
    'Total Restaurants': len(df_clean),
    'Unique Locations': df_clean['location'].nunique(),
    'Unique Cuisines': df_clean['primary_cuisine'].nunique(),
    'Avg Rating': df_clean['rate_numeric'].mean(),
    'Avg Cost for Two': df_clean['cost_for_two'].mean(),
    'Avg Votes': df_clean['votes'].mean(),
    'Online Order Available': f"{(df_clean['has_online_order'].mean() * 100):.1f}%",
    'Table Booking Available': f"{(df_clean['has_table_booking'].mean() * 100):.1f}%"
}

for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:,.2f}")
    else:
        print(f"  {key}: {value}")

# -------- 5.2: Rating Analysis --------
print("\n\n5.2 Rating Distribution Analysis")
print("-" * 40)

rating_stats = df_clean['rate_numeric'].describe()
print(rating_stats)

rating_categories = pd.cut(df_clean['rate_numeric'], 
                          bins=[0, 2.5, 3.5, 4.0, 5.0],
                          labels=['Poor (0-2.5)', 'Average (2.5-3.5)', 'Good (3.5-4.0)', 'Excellent (4.0-5.0)'])
rating_dist = rating_categories.value_counts().sort_index()
print("\nRating Categories:")
for cat, count in rating_dist.items():
    pct = (count / rating_dist.sum()) * 100
    print(f"  {cat}: {count:,} ({pct:.1f}%)")

STEP 4: EXPLORATORY DATA ANALYSIS
--------------------------------------------------------------------------------

5.1 Overall Dataset Statistics
----------------------------------------
  Total Restaurants: 51717
  Unique Locations: 93
  Unique Cuisines: 88
  Avg Rating: 3.70
  Avg Cost for Two: 555.43
  Avg Votes: 283.70
  Online Order Available: 58.9%
  Table Booking Available: 12.5%


5.2 Rating Distribution Analysis
----------------------------------------
count    41665.000000
mean         3.700449
std          0.440513
min          1.800000
25%          3.400000
50%          3.700000
75%          4.000000
max          4.900000
Name: rate_numeric, dtype: float64

Rating Categories:
  Poor (0-2.5): 288 (0.7%)
  Average (2.5-3.5): 13,996 (33.6%)
  Good (3.5-4.0): 18,165 (43.6%)
  Excellent (4.0-5.0): 9,216 (22.1%)


In [53]:
# -------- 5.3: Cost Analysis --------
print("\n\n5.3 Cost Distribution Analysis")
print("-" * 40)

cost_stats = df_clean['cost_for_two'].describe()
print(cost_stats)

cost_categories = pd.cut(df_clean['cost_for_two'],
                        bins=[0, 300, 600, 1000, 10000],
                        labels=['Budget (<300)', 'Mid-Range (300-600)', 'Premium (600-1000)', 'Luxury (>1000)'])
cost_dist = cost_categories.value_counts().sort_index()
print("\nPrice Categories:")
for cat, count in cost_dist.items():
    pct = (count / cost_dist.sum()) * 100
    print(f"  {cat}: {count:,} ({pct:.1f}%)")

# -------- 5.4: Location Analysis --------
print("\n\n5.4 Top Locations by Restaurant Count")
print("-" * 40)

top_locations = df_clean['location'].value_counts().head(15)
print(top_locations)





5.3 Cost Distribution Analysis
----------------------------------------
count    51371.000000
mean       555.431566
std        438.850728
min         40.000000
25%        300.000000
50%        400.000000
75%        650.000000
max       6000.000000
Name: cost_for_two, dtype: float64

Price Categories:
  Budget (<300): 18,554 (36.1%)
  Mid-Range (300-600): 19,205 (37.4%)
  Premium (600-1000): 8,332 (16.2%)
  Luxury (>1000): 5,280 (10.3%)


5.4 Top Locations by Restaurant Count
----------------------------------------
location
BTM                      5124
HSR                      2523
Koramangala 5th Block    2504
JP Nagar                 2235
Whitefield               2144
Indiranagar              2083
Jayanagar                1926
Marathahalli             1846
Bannerghatta Road        1630
Bellandur                1286
Electronic City          1258
Koramangala 1st Block    1238
Brigade Road             1218
Koramangala 7th Block    1181
Koramangala 6th Block    1156
Name: count, dtype

In [49]:

# -------- 5.5: Cuisine Analysis --------
print("\n\n5.5 Top Cuisines Analysis")
print("-" * 40)

top_cuisines = df_clean['primary_cuisine'].value_counts().head(15)
print(top_cuisines)

# -------- 5.6: Restaurant Type Analysis --------
print("\n\n5.6 Restaurant Type Distribution")
print("-" * 40)

rest_types = df_clean['rest_type'].value_counts().head(10)
print(rest_types)




5.5 Top Cuisines Analysis
----------------------------------------
primary_cuisine
North Indian    12299
South Indian     5010
Cafe             4330
Chinese          3066
Biryani          3055
Fast Food        2592
Bakery           2175
Desserts         2150
Continental      1846
Beverages        1255
Andhra           1204
Ice Cream         918
Kerala            887
Street Food       841
Mithai            716
Name: count, dtype: int64


5.6 Restaurant Type Distribution
----------------------------------------
rest_type
Quick Bites           19132
Casual Dining         10330
Cafe                   3732
Delivery               2604
Dessert Parlor         2263
Takeaway, Delivery     2037
Casual Dining, Bar     1154
Bakery                 1141
Beverage Shop           867
Bar                     697
Name: count, dtype: int64


In [57]:
# -------- 5.7: Popular Dishes Analysis --------
print("\n\n5.7 Most Popular Dishes")
print("-" * 40)

all_dishes = []
for dishes in df_clean['dishes_list']:
    all_dishes.extend(dishes)

dish_counter = Counter(all_dishes)
top_dishes = dish_counter.most_common(20)
print("\nTop 20 Popular Dishes:")
for i, (dish, count) in enumerate(top_dishes, 1):
    print(f"  {i}. {dish}: {count:,} mentions")



5.7 Most Popular Dishes
----------------------------------------

Top 20 Popular Dishes:
  1. Pasta: 3,409 mentions
  2. Burgers: 3,024 mentions
  3. Cocktails: 2,832 mentions
  4. Pizza: 2,737 mentions
  5. Biryani: 2,092 mentions
  6. Coffee: 1,994 mentions
  7. Mocktails: 1,887 mentions
  8. Sandwiches: 1,676 mentions
  9. Paratha: 1,566 mentions
  10. Noodles: 1,415 mentions
  11. Salads: 1,391 mentions
  12. Fish: 1,368 mentions
  13. Nachos: 1,365 mentions
  14. Chicken Biryani: 1,330 mentions
  15. Beer: 1,268 mentions
  16. Mutton Biryani: 1,100 mentions
  17. Tea: 1,039 mentions
  18. Thali: 1,004 mentions
  19. Rolls: 991 mentions
  20. Waffles: 979 mentions


In [59]:
print("STEP 5: ADVANCED ANALYTICS & INSIGHTS")
print("-" * 80)

# -------- 6.1: Rating vs Cost Correlation --------
print("\n6.1 Correlation Analysis")
print("-" * 40)

# Select numeric columns for correlation
numeric_cols = ['rate_numeric', 'cost_for_two', 'votes', 'cuisine_count', 
                'has_online_order', 'has_table_booking', 'dish_count']
correlation_data = df_clean[numeric_cols].corr()['rate_numeric'].sort_values(ascending=False)
print("\nCorrelation with Rating:")
print(correlation_data)

# -------- 6.2: Best Rated Restaurants --------
print("\n\n6.2 Top 10 Highest Rated Restaurants (by votes)")
print("-" * 40)

top_rated = df_clean[df_clean['votes'] > 100].nlargest(10, 'rate_numeric')[
    ['name', 'location', 'rate_numeric', 'votes', 'cost_for_two', 'cuisines']
]
print(top_rated.to_string(index=False))


STEP 5: ADVANCED ANALYTICS & INSIGHTS
--------------------------------------------------------------------------------

6.1 Correlation Analysis
----------------------------------------

Correlation with Rating:
rate_numeric         1.000000
dish_count           0.603556
votes                0.434040
has_table_booking    0.424631
cost_for_two         0.385183
cuisine_count        0.206729
has_online_order     0.068479
Name: rate_numeric, dtype: float64


6.2 Top 10 Highest Rated Restaurants (by votes)
----------------------------------------
                          name              location  rate_numeric  votes  cost_for_two                                                      cuisines
   Byg Brewski Brewing Company         Sarjapur Road           4.9  16345        1600.0 Continental, North Indian, Italian, South Indian, Finger Food
   Byg Brewski Brewing Company         Sarjapur Road           4.9  16345        1600.0 Continental, North Indian, Italian, South Indian, Finger Food
  

In [67]:
# -------- 6.3: Most Popular Restaurants (by votes) --------
print("\n\n6.3 Top 10 Most Popular Restaurants (by votes)")
print("-" * 40)

most_popular = df_clean.nlargest(10, 'votes')[
    ['name', 'location', 'votes', 'rate_numeric', 'cost_for_two']
]
print(most_popular.to_string(index=False))

# -------- 6.4: Best Value Restaurants --------
print("\n\n6.4 Top 10 Best Value Restaurants (High Rating, Low Cost)")
print("-" * 40)

# Calculate value score: Rating / Cost (normalized)
df_clean['value_score'] = df_clean['rate_numeric'] / (df_clean['cost_for_two'] + 1) * 1000
best_value = df_clean[(df_clean['rate_numeric'] >= 4.0) & (df_clean['cost_for_two'] <= 500)].nlargest(10, 'value_score')[
    ['name', 'location', 'rate_numeric', 'cost_for_two', 'cuisines']
]
print(best_value.to_string(index=False))



6.3 Top 10 Most Popular Restaurants (by votes)
----------------------------------------
                       name              location  votes  rate_numeric  cost_for_two
Byg Brewski Brewing Company         Sarjapur Road  16832           4.9        1600.0
Byg Brewski Brewing Company         Sarjapur Road  16832           4.9        1600.0
Byg Brewski Brewing Company         Sarjapur Road  16832           4.9        1600.0
Byg Brewski Brewing Company         Sarjapur Road  16345           4.9        1600.0
Byg Brewski Brewing Company         Sarjapur Road  16345           4.9        1600.0
Byg Brewski Brewing Company         Sarjapur Road  16345           4.9        1600.0
                       Toit           Indiranagar  14956           4.7        1500.0
                       Toit           Indiranagar  14956           4.7        1500.0
                   Truffles Koramangala 5th Block  14726           4.7         900.0
                   Truffles Koramangala 5th Block  14723    

In [73]:
# -------- 6.5: Location-wise Performance --------
print("\n\n6.5 Top Locations by Average Rating")
print("-" * 40)

location_stats = df_clean.groupby('location').agg({
    'rate_numeric': 'mean',
    'cost_for_two': 'mean',
    'votes': 'sum',
    'name': 'count'
}).rename(columns={'name': 'restaurant_count'})

location_stats = location_stats[location_stats['restaurant_count'] >= 10].sort_values('rate_numeric', ascending=False).head(10)
print(location_stats.round(2))

# -------- 6.6: Cuisine Performance Analysis --------
print("\n\n6.6 Top Cuisines by Average Rating")
print("-" * 40)

cuisine_stats = df_clean.groupby('primary_cuisine').agg({
    'rate_numeric': 'mean',
    'cost_for_two': 'mean',
    'votes': 'sum',
    'name': 'count'
}).rename(columns={'name': 'restaurant_count'})

cuisine_stats = cuisine_stats[cuisine_stats['restaurant_count'] >= 20].sort_values('rate_numeric', ascending=False).head(10)
print(cuisine_stats.round(2))




6.5 Top Locations by Average Rating
----------------------------------------
                       rate_numeric  cost_for_two    votes  restaurant_count
location                                                                    
Lavelle Road                   4.14       1307.93   506186               529
Koramangala 3rd Block          4.02        778.47   125159               216
St. Marks Road                 4.02        871.31   266099               352
Koramangala 5th Block          4.01        663.66  2219506              2504
Church Street                  3.99        834.89   594979               569
Sankey Road                    3.97       2505.56     6411                27
Koramangala 4th Block          3.92        696.36   685156              1017
Cunningham Road                3.90        864.97   287873               491
Residency Road                 3.86        966.32   291954               675
MG Road                        3.86       1155.70   432111               9

In [75]:
# -------- 6.7: Online Order vs Dine-in Analysis --------
print("\n\n6.7 Online Order vs Dine-in Comparison")
print("-" * 40)

online_comparison = df_clean.groupby('has_online_order').agg({
    'rate_numeric': 'mean',
    'cost_for_two': 'mean',
    'votes': 'mean',
    'name': 'count'
}).rename(columns={'name': 'count'})
online_comparison.index = ['Dine-in Only', 'Online Order Available']
print(online_comparison.round(2))

# -------- 6.8: Table Booking Analysis --------
print("\n\n6.8 Table Booking Impact Analysis")
print("-" * 40)

booking_comparison = df_clean.groupby('has_table_booking').agg({
    'rate_numeric': 'mean',
    'cost_for_two': 'mean',
    'votes': 'mean',
    'name': 'count'
}).rename(columns={'name': 'count'})
booking_comparison.index = ['No Table Booking', 'Table Booking Available']
print(booking_comparison.round(2))



6.7 Online Order vs Dine-in Comparison
----------------------------------------
                        rate_numeric  cost_for_two   votes  count
Dine-in Only                    3.66        599.46  250.32  21273
Online Order Available          3.72        525.16  307.02  30444


6.8 Table Booking Impact Analysis
----------------------------------------
                         rate_numeric  cost_for_two    votes  count
No Table Booking                 3.62        452.66   160.69  45268
Table Booking Available          4.14       1271.34  1147.13   6449
